# This is the official training Illustration of PuzzleTuning
* Use google colab pro+ (high RAM+GPU) to run 24 hours
* we use the Python3.7 Pytorch 1.9.0+cu111 torchvision 0.10.0+cu111
* we use the A100 GPU for the data-flow illustration with Colab

The code and Training process along with all record are Open-Source:
* PuzzleTuning official github page: https://github.com/sagizty/PuzzleTuning
* The dataset CPIA is publicly aviliable at: https://github.com/zhanglab2021/CPIA_Dataset


## Check Colab GPU

In [1]:
# check GPU
!nvidia-smi

Fri Nov 24 06:31:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    55W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!date --date='+8 hour'  # CST time zone

Fri Nov 24 02:31:52 PM UTC 2023


## Mount Google Drive

This will save output images to your google drive, you can remove this line and the last part if you don't want the output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Build file-system enviroment

In [ ]:
# create file-system enviroment
# mount the google drive first
# https://drive.google.com/drive/u/1/my-drive

# clear colab path
!rm -rf /data
!rm -rf /home/Pathology_Experiment

# create path
!mkdir /home/Pathology_Experiment
!mkdir /home/Pathology_Experiment/runs
!mkdir /home/Pathology_Experiment/code
!mkdir /home/Pathology_Experiment/saved_models
!mkdir /home/Pathology_Experiment/imaging_results

!mkdir /data
!mkdir /data/Pathology_Experiment
!mkdir /data/Pathology_Experiment/dataset

print('Folder Tree Creation completed!')

# get latest code from Github pancreatic-cancer-diagnosis-tansformer page
!git clone https://github.com/sagizty/PuzzleTuning.git /home/Pathology_Experiment/code
print('code transfer from github completed!')

# get the CLS dataset by its zip
!mv /home/Pathology_Experiment/code/Archive/* /data/Pathology_Experiment/dataset/
# unzip
!unzip -q /data/Pathology_Experiment/dataset/PuzzleTuning_demoset.zip -d /data/Pathology_Experiment/dataset/
!unzip -q /data/Pathology_Experiment/dataset/warwick_CLS.zip -d /data/Pathology_Experiment/dataset/
# alter the path
!rm -f /data/Pathology_Experiment/dataset/PuzzleTuning_demoset.zip
!rm -f /data/Pathology_Experiment/dataset/warwick_CLS.zip
print('data transfer completed!')

## Arrange the working enviorment

In [ ]:
# get packages
!pip install tqdm
!pip install timm==0.5.4
!pip install einops
!pip install ml_collections
!pip install ttach
!pip install notifyemail
!pip install psutil
!pip install scipy
!pip install torchsummary
!pip install tensorboardX
!pip install opencv_contrib_python
!pip install matplotlib
!pip install ipykernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ml_collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94507 sha256=b01d6551e051eb94b398f73bab7144889d7ee2a44f392da831939d3029f7119a
  Stored in directory: /root/.cache/pip/wheels/7b/89/c9/a9b87790789e94aadcfc393c283e3ecd5ab916aed0a31be8fe
Successfully built ml_collections
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.2 MB/s eta 0:00:00


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip list
!pip freeze>requirements.txt
!cp requirements.txt ../runs

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.6
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b

# Pre-Training
* set up path by command line
* use argparse to set down hyper-parameter

10000epochs will be trined with 400 images, for data-flow illustration.
We suggest you to use 4 * A100 SMX4 GPUs to train PuzzleTuning with CPIA dataset.



Our official training script is given here:

In [ ]:
# nohup python PuzzleTuning.py --batch_size 64 --group_shuffle_size 16 --blr 1.5e-4 --epochs 200 --accum_iter 2 --print_freq 2000 --check_point_gap 50 --input_size 224 --warmup_epochs 20 --pin_mem --num_workers 32 --strategy loop --PromptTuning Deep --basic_state_dict ../saved_models/ViT_b16_224_Imagenet.pth --data_path ../datasets/All &

All following lines are for data-flow illustation with colab

In [ ]:
# change working dir
import os
os.chdir("/home/Pathology_Experiment/code")
!pwd

/home/Pathology_Experiment/code


Training

In [ ]:
!python PuzzleTuning.py --model sae_vit_base_patch16 --PromptTuning Deep --batch_size 32 --group_shuffle_size 8 --strategy loop --blr 1.5e-5 --epochs 40000 --warmup_epochs 20 --accum_iter 2 --print_freq 200 --check_point_gap 10000 --input_size 224 --pin_mem --num_workers 2 --basic_state_dict timm --data_path /data/Pathology_Experiment/dataset/PuzzleTuning_demoset --output_dir /home/Pathology_Experiment/runs --log_dir /home/Pathology_Experiment/imaging_results

Streaming output truncated to the last 5000 lines.
Figure(640x480)
log_dir: /home/Pathology_Experiment/imaging_results/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20
Epoch: [39167]  [ 0/12]  eta: 0:00:03  lr: 0.000000  loss: 0.0216 (0.0216)  time: 0.2915  data: 0.2048  max mem: 5716
Epoch: [39167]  [11/12]  eta: 0:00:00  lr: 0.000000  loss: 0.0247 (0.0241)  time: 0.0953  data: 0.0172  max mem: 5716
Epoch: [39167] Total time: 0:00:01 (0.0994 s / it)
Averaged stats: lr: 0.000000  loss: 0.0247 (0.0241) fix_position_ratio: 0.25   puzzle_patch_size: 112
Figure(640x480)
log_dir: /home/Pathology_Experiment/imaging_results/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20
Epoch: [39168]  [ 0/12]  eta: 0:00:03  lr: 0.000000  loss: 0.0211 (0.0211)  time: 0.2886  data: 0.1982  max mem: 5716
Epoch: [39168]  [11/12]  eta: 0:00:00  lr: 0.000000  loss: 0.0240 (0.0244)  time: 0.0966  data: 0.0167  max mem: 5716
Epoch: [39168] Total time: 0:00:01 (0.1007 s / it)
Averaged stats: lr: 0

Visulization

In [ ]:
!python PuzzleTesting.py --model sae_vit_base_patch16 --PromptTuning Deep --Prompt_Token_num 20 --batch_size 8 --fix_position_ratio 0.5 --fix_patch_size 16 --enable_visualize_check --data_path /data/Pathology_Experiment/dataset/PuzzleTuning_demoset --output_dir /home/Pathology_Experiment/imaging_results --log_dir /home/Pathology_Experiment/imaging_results --checkpoint_path /home/Pathology_Experiment/runs/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20_checkpoint-39999.pth



Testing_PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20_b_8_hint_ratio_0.5_patch_size_16


Use 1 GPUs of idx: 0
job dir: /home/Pathology_Experiment/code
Namespace(model_idx='Testing_PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20_b_8_hint_ratio_0.5_patch_size_16',
batch_size=8,
model='sae_vit_base_patch16',
seg_decoder=None,
input_size=224,
num_classes=3,
mask_ratio=None,
fix_position_ratio=0.5,
fix_patch_size=16,
group_shuffle_size=-1,
shuffle_dataloader=False,
PromptTuning='Deep',
Prompt_Token_num=20,
norm_pix_loss=False,
data_path='/data/Pathology_Experiment/dataset/PuzzleTuning_demoset',
output_dir='/home/Pathology_Experiment/imaging_results',
log_dir='/home/Pathology_Experiment/imaging_results',
gpu_idx=0,
device='cuda',
seed=42,
checkpoint_path='/home/Pathology_Experiment/runs/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20_checkpoint-39999.pth',
combined_pred_illustration=False,
enable_visu

Load-up the ViT prompt weight from pre-trained checkpoint

In [ ]:
os.chdir("/home/Pathology_Experiment/code/utils")

In [ ]:
!python transfermodel.py --given_name ViT_b16_224_timm_PuzzleTuning_SAE_CPIAm_Prompt_Deep_tokennum_20_promptstate.pth --model_idx ViT --PromptTuning Deep --Prompt_Token_num 20 --edge_size 224 --checkpoint_path /home/Pathology_Experiment/runs/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20/PuzzleTuning_sae_vit_base_patch16_Prompt_Deep_tokennum_20_checkpoint-39999.pth --save_model_path /home/Pathology_Experiment/saved_models

model forward cheacked
model is ready now!
checkpoint epoch 39999
DataParallel model loaded
model : ViT
prompt obtained
model trained by multi-GPUs has its single GPU copy saved at  /home/Pathology_Experiment/saved_models/ViT_b16_224_timm_PuzzleTuning_SAE_CPIAm_Prompt_Deep_tokennum_20_promptstate.pth


# Finetuning and comparison
* set up path by command line
* use argparse to set down hyper-parameter

## Finetuning without PuzzleTuning

In [ ]:
os.chdir("/home/Pathology_Experiment/code")

### ViT (with timm weight)

Train

In [ ]:
!python Train.py --edge_size 224 --data_augmentation_mode 3 --lr 1e-05 --lrf 0.15 --enable_tensorboard --model_idx ViT_base_timm_401_lf15_finetuning_warwick_CLS --dataroot /data/Pathology_Experiment/dataset/warwick_CLS --draw_root /home/Pathology_Experiment/runs/ViT_finetuning_with_timm --model_path /home/Pathology_Experiment/saved_models

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(model_idx='ViT_base_timm_401_lf15_finetuning_warwick_CLS', drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0, cls_token_off=False, pos_embedding_off=False, att_module='SimAM', backbone_PT_off=False, gpu_idx=-1, dataroot='/data/Pathology_Experiment/dataset/warwick_CLS', model_path='/home/Pathology_Experiment/saved_models', draw_root='/home/Pathology_Experiment/runs/ViT_finetuning_with_timm', paint=True, enable_tensorboard=True, enable_attention_check=False, enable_visualize_check=False, PromptTuning=None, Prompt_Token_num=20, PromptUnFreeze=False, linearprobing=False, Pre_Trained_model_path=None, Prompt_state_path=None, enable_sam=False, augmentation_name=None, ratio_strategy=None, patch_strategy=None, loss_drive_threshold=4.0, fix_position_ratio=0.5, fix_patch_size=None, patch_size_jump=None, num_classes=0, edge_size=224, data_augmentation_mode=3, batch_size=

Test

In [ ]:
!python Test.py --edge_size 224 --data_augmentation_mode 3 --model_idx ViT_base_timm_401_lf15_finetuning_warwick_CLS --dataroot /data/Pathology_Experiment/dataset/warwick_CLS --draw_root /home/Pathology_Experiment/runs/ViT_finetuning_with_timm --model_path /home/Pathology_Experiment/saved_models

class_names: ['benign', 'malignant']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_384',
 'vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',

### VPT + finetuning (with timm weight)

Train

In [ ]:
!python Train.py --edge_size 224 --data_augmentation_mode 3 --lr 1e-05 --lrf 0.15 --enable_tensorboard --model_idx ViT_base_timm_PromptDeep_20_401_lf15_finetuning_warwick_CLS --PromptTuning Deep --dataroot /data/Pathology_Experiment/dataset/warwick_CLS --draw_root /home/Pathology_Experiment/runs/VPT_finetuning_with_timm --model_path /home/Pathology_Experiment/saved_models

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(model_idx='ViT_base_timm_PromptDeep_20_401_lf15_finetuning_warwick_CLS', drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0, cls_token_off=False, pos_embedding_off=False, att_module='SimAM', backbone_PT_off=False, gpu_idx=-1, dataroot='/data/Pathology_Experiment/dataset/warwick_CLS', model_path='/home/Pathology_Experiment/saved_models', draw_root='/home/Pathology_Experiment/runs/VPT_finetuning_with_timm', paint=True, enable_tensorboard=True, enable_attention_check=False, enable_visualize_check=False, PromptTuning='Deep', Prompt_Token_num=20, PromptUnFreeze=False, linearprobing=False, Pre_Trained_model_path=None, Prompt_state_path=None, enable_sam=False, augmentation_name=None, ratio_strategy=None, patch_strategy=None, loss_drive_threshold=4.0, fix_position_ratio=0.5, fix_patch_size=None, patch_size_jump=None, num_classes=0, edge_size=224, data_augmentation_mod

Test

In [ ]:
!python Test.py --edge_size 224 --data_augmentation_mode 3 --model_idx ViT_base_timm_PromptDeep_20_401_lf15_finetuning_warwick_CLS --PromptTuning Deep --dataroot /data/Pathology_Experiment/dataset/warwick_CLS --draw_root /home/Pathology_Experiment/runs/VPT_finetuning_with_timm --model_path /home/Pathology_Experiment/saved_models

class_names: ['benign', 'malignant']
base_state_dict of timm
Test the PromptTuning of  ViT_base_timm_PromptDeep_20_401_lf15_finetuning_warwick_CLS
Prompt VPT type: Deep
in prompt model building, timm ViT loaded for base_state_dict
model forward cheacked
model is ready now!
prompt head match
model loaded
model : ViT_base_timm_PromptDeep_20_401_lf15_finetuning_warwick_CLS
*********************************setting*************************************
Namespace(model_idx='ViT_base_timm_PromptDeep_20_401_lf15_finetuning_warwick_CLS', drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0, cls_token_off=False, pos_embedding_off=False, att_module='SimAM', gpu_idx=0, dataroot='/data/Pathology_Experiment/dataset/warwick_CLS', model_path='/home/Pathology_Experiment/saved_models', draw_root='/home/Pathology_Experiment/runs/VPT_finetuning_with_timm', model_path_by_hand=None, paint=True, enable_tensorboard=False, enable_attention_check=False, enable_visualize_check=False, data_augmentation_mode=3, Pr

## Finetuning with PuzzleTuning Prompt
VPT + finetuning (with timm weight & PuzzleTuning Prompt)

Train

In [ ]:
!python Train.py --edge_size 224 --data_augmentation_mode 3 --lr 1e-05 --lrf 0.15 --enable_tensorboard --model_idx ViT_base_timm_PuzzleTuning_SAE_promptstate_PromptDeep_20_401_lf15_finetuning_warwick_CLS --PromptTuning Deep --Prompt_Token_num 20 --PromptUnFreeze --dataroot /data/Pathology_Experiment/dataset/warwick_CLS --draw_root /home/Pathology_Experiment/runs/SAE-timm-start_promptstate_VPT_finetuning_with_timm --Prompt_state_path /home/Pathology_Experiment/saved_models/ViT_b16_224_timm_PuzzleTuning_SAE_CPIAm_Prompt_Deep_tokennum_20_promptstate.pth --model_path /home/Pathology_Experiment/saved_models

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(model_idx='ViT_base_timm_PuzzleTuning_SAE_promptstate_PromptDeep_20_401_lf15_finetuning_warwick_CLS', drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0, cls_token_off=False, pos_embedding_off=False, att_module='SimAM', backbone_PT_off=False, gpu_idx=-1, dataroot='/data/Pathology_Experiment/dataset/warwick_CLS', model_path='/home/Pathology_Experiment/saved_models', draw_root='/home/Pathology_Experiment/runs/SAE-timm-start_promptstate_VPT_finetuning_with_timm', paint=True, enable_tensorboard=True, enable_attention_check=False, enable_visualize_check=False, PromptTuning='Deep', Prompt_Token_num=20, PromptUnFreeze=True, linearprobing=False, Pre_Trained_model_path=None, Prompt_state_path='/home/Pathology_Experiment/saved_models/ViT_b16_224_timm_PuzzleTuning_SAE_CPIAm_Prompt_Deep_tokennum_20_promptstate.pth', enable_sam=False, augmentation_name=None, ratio_strategy

Test

In [ ]:
!python Test.py --edge_size 224 --data_augmentation_mode 3 --model_idx ViT_base_timm_PuzzleTuning_SAE_promptstate_PromptDeep_20_401_lf15_finetuning_warwick_CLS --PromptTuning Deep --Prompt_Token_num 20 --PromptUnFreeze --dataroot /data/Pathology_Experiment/dataset/warwick_CLS --draw_root /home/Pathology_Experiment/runs/SAE-timm-start_promptstate_VPT_finetuning_with_timm --model_path /home/Pathology_Experiment/saved_models

class_names: ['benign', 'malignant']
base_state_dict of timm
Test the PromptTuning of  ViT_base_timm_PuzzleTuning_SAE_promptstate_PromptDeep_20_401_lf15_finetuning_warwick_CLS
Prompt VPT type: Deep
in prompt model building, timm ViT loaded for base_state_dict
model forward cheacked
model is ready now!
model loaded
model : ViT_base_timm_PuzzleTuning_SAE_promptstate_PromptDeep_20_401_lf15_finetuning_warwick_CLS
*********************************setting*************************************
Namespace(model_idx='ViT_base_timm_PuzzleTuning_SAE_promptstate_PromptDeep_20_401_lf15_finetuning_warwick_CLS', drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0, cls_token_off=False, pos_embedding_off=False, att_module='SimAM', gpu_idx=0, dataroot='/data/Pathology_Experiment/dataset/warwick_CLS', model_path='/home/Pathology_Experiment/saved_models', draw_root='/home/Pathology_Experiment/runs/SAE-timm-start_promptstate_VPT_finetuning_with_timm', model_path_by_hand=None, paint=True, enable_tensorboard

# check the Tensorboard output

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/home/Pathology_Experiment/runs'

<IPython.core.display.Javascript object>

# After the task, save the output to google drive


In [ ]:
# change working dir
import os
os.chdir("/home/Pathology_Experiment/code/utils")
!python check_log_json.py --enable_notify --draw_root /home/Pathology_Experiment/runs --record_dir /home/Pathology_Experiment/CSV_logs

usage: check_log_json.py [-h] [--ONE_LOG] [--draw_root DRAW_ROOT] [--record_dir RECORD_DIR]
check_log_json.py: error: unrecognized arguments: --enable_notify


In [ ]:
# copy tensorboard runs
!/bin/cp -rf /home/Pathology_Experiment/runs/*  /content/drive/MyDrive/Pathology_Experiment/runs/
print('runs copy completed!')
# copy the traind models
!/bin/cp -rf /home/Pathology_Experiment/saved_models/* /content/drive/MyDrive/Pathology_Experiment/saved_models/
print('models copy completed!')
# copy the imaging_results
!/bin/cp -rf /home/Pathology_Experiment/imaging_results/* /content/drive/MyDrive/Pathology_Experiment/imaging_results/
print('imaging_results copy completed!')

/bin/cp: target '/content/drive/MyDrive/Pathology_Experiment/runs/' is not a directory
runs copy completed!
/bin/cp: target '/content/drive/MyDrive/Pathology_Experiment/saved_models/' is not a directory
models copy completed!
/bin/cp: target '/content/drive/MyDrive/Pathology_Experiment/imaging_results/' is not a directory
imaging_results copy completed!


In [ ]:
!date --date='+8 hour'  # CST time zone

Sat Nov 25 08:22:43 AM UTC 2023
